## Imports

In [1]:
import glob
import os
import pickle
import pprint

import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score

from catboost import CatBoostClassifier

from utils.dataset_maker import connected_subset, ContrastiveDataset, GraphAugmentation, train_test_val_split

from IPython.display import clear_output

Using backend: pytorch


In [2]:
DATA_DIR = "data"
INTERIM_DIR = "interim"
MODEL = "models"

In [3]:
pp = pprint.PrettyPrinter(indent=4)

## Load data

We download the predefined dataset in its grpah form with networkx library. For this particular experiment we don't need to modify data, so we describe this explicitly.

In [4]:
dataset_fp = os.path.join("..", DATA_DIR, INTERIM_DIR, "dataset_for_semisupervised_1hop")
augmentations = [GraphAugmentation('identical')]
dataset = ContrastiveDataset(
    filename=dataset_fp,
    augmentations=augmentations,
)
c_dataset = connected_subset(dataset)
n = len(c_dataset)
n

285

We also repeat some steps of graph creation to collect all the nodes, parsed previously.

In [5]:
train_test = []
labels_ = []
csv_fp = os.path.join("..", DATA_DIR, INTERIM_DIR, "nodes_0", "*.csv")
for file in glob.glob(csv_fp):
    df = pd.read_csv(file, index_col="Unnamed: 0").drop_duplicates().astype(int)
    train_test.append(df.drop("n_groups", axis=1).iloc[0])
    l = file[-5]
    labels_.append(int(l))

In [6]:
np.random.seed(0)
ratio = [0.6, 0.2, 0.2]
split_idx = ['train'] * int(ratio[0] * n) \
    + ['test'] * int(ratio[1] * n) \
    + ['val'] * int(ratio[2] * n)
split_idx = np.random.permutation(split_idx)
train_idx = np.where(split_idx == 'train')[0]
test_idx = np.where(split_idx == 'test')[0]
val_idx = np.where(split_idx == 'val')[0]

In [7]:
X_train = np.array(train_test)[train_idx]
X_test = np.array(train_test)[test_idx]
X_val = np.array(train_test)[val_idx]
*_, y_train, y_test, y_val = train_test_val_split(
    c_dataset, train_idx, test_idx, val_idx
)
(len(X_train), len(X_test), len(X_val)), (len(y_train), len(y_test), len(y_val))

((171, 57, 57), (171, 57, 57))

## Models

Here we test several classic ML approaches such logistic regression, random forests and boosting. We compare the metrics and save the best performing model for further possible use.

#### Logistic Regression

In [8]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
score = lr.score(X_test, y_test)
y_pred = lr.predict(X_val)
print("True values:")
pp.pprint(y_val)
print("Prediction:")
pp.pprint(y_pred)
prec = precision_score(y_pred, y_val)
recall = recall_score(y_pred, y_val)
f1 = f1_score(y_pred, y_val)
acc = accuracy_score(y_pred, y_val)
print("Precision: ", prec)
print("Recall   : ", recall)
print("F1       : ", f1)
print("Score    : ", score)
print("Accuracy : ", acc)

True values:
tensor([1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1.,
        1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
        0., 1., 1.])
Prediction:
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1.], dtype=float32)
Precision:  1.0
Recall   :  0.6842105263157895
F1       :  0.8125000000000001
Score    :  0.631578947368421
Accuracy :  0.6842105263157895


#### Random Forest Classifier

In [9]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
score = rfc.score(X_test, y_test)
y_pred = rfc.predict(X_val)
print("True values:")
pp.pprint(y_val)
print("Prediction:")
pp.pprint(y_pred)
prec = precision_score(y_pred, y_val)
recall = recall_score(y_pred, y_val)
f1 = f1_score(y_pred, y_val)
acc = accuracy_score(y_pred, y_val)
rocauc = roc_auc_score(y_pred, y_val)
print("Precision: ", prec)
print("Recall   : ", recall)
print("F1       : ", f1)
print("Score    : ", score)
print("Accuracy : ", acc)
print("ROCAUC   : ", rocauc)

True values:
tensor([1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1.,
        1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
        0., 1., 1.])
Prediction:
array([1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0.,
       1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1.,
       1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 0., 1.], dtype=float32)
Precision:  0.9743589743589743
Recall   :  0.926829268292683
F1       :  0.9500000000000001
Score    :  0.9122807017543859
Accuracy :  0.9298245614035088
ROCAUC   :  0.9321646341463414


#### CatBoost Classifier

In [10]:
cb = CatBoostClassifier(verbose=False)
cb.fit(X_train, y_train.numpy(), plot=True)
score = cb.score(X_test, y_test.numpy())
y_pred = cb.predict(X_val)
print("True values:")
pp.pprint(y_val)
print("Prediction:")
pp.pprint(y_pred)
prec = precision_score(y_pred, y_val)
recall = recall_score(y_pred, y_val)
f1 = f1_score(y_pred, y_val)
acc = accuracy_score(y_pred, y_val)
rocauc = roc_auc_score(y_pred, y_val)
print("Precision: ", prec)
print("Recall   : ", recall)
print("F1       : ", f1)
print("Score    : ", score)
print("Accuracy : ", acc)
print("ROCAUC   : ", rocauc)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

True values:
tensor([1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1.,
        1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
        0., 1., 1.])
Prediction:
array([1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0.,
       1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1.,
       1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1.])
Precision:  1.0
Recall   :  0.9285714285714286
F1       :  0.962962962962963
Score    :  0.9122807017543859
Accuracy :  0.9473684210526315
ROCAUC   :  0.9642857142857143


So, for further use we save the CatBoost model, which performed the best f1 and ROCAUC so far.

In [11]:
model_fp = os.path.join("..", MODEL, "cb_model")
cb.save_model(model_fp)